In [ ]:
# !pip3 install yahooquery
# !pip3 install yfinance
# !pip3 install arch

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import yfinance as yf

from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from pprint import pprint
from tqdm import tqdm

In [10]:
# predetermined lists of ETFs
COUNTRY_ETF_TICKERS = [
    'EWJ','MCHI','BBJP','VPL','KWEB','FXI',
    'BBAX','EWT','EWY','AAXJ','ASHR',
    'EWA','EPP','IPAC','DXJ','AIA','GXC',
    'FLJP','CQQQ','EWH','EPI','CXSE','INDY',
    'EEMA','EWS','VNM','KBA','EIDO','GMF',
    'THD','CHIQ','CNYA','FLKR','SMIN','DBJP',
    'EWJV','EWM','PGJ','DFJ','MINV','ENZL',
    'HEWJ','EPHE','FLCH',
]

SECTOR_ETF_TICKERS = [
    'VGT','XLK','XLE','XLV','XLF','XLP','VHT','XLU','XLI',
    'XLY','GDX','IYW','XLC','VFH','VDE','IBB','SMH','SOXX',
    'ARKK','VDC','AMLP','XBI','ITA','KWEB','IHI','XLB','FTEC',
    'VPU','ICLN','IGV','CIBR','PAVE','VCR','IXJ','VIS','IGF',
    'XOP','GNR','FDN','GDXJ','LIT','IYH','XT','FHLC','VAW',
    'IXN','AMJ','IGM','SKYY','OIH','VOX','EMLP','RYT','TAN',
    'XME','NFRA','KRE','FUTY','JETS','FXO','IXC','IYF','PPA',
    'ARKG','IFRA','COPX','KBWB','FXZ','PHO','IYE','IYK','FXR',
    'QCLN','EUFN','KOMP','BOTZ','PICK','TDIV','FENY','FXH','ITB',
    'XAR','QTEC','KXI','FNCL','FBT','URA','MLPA','HACK','KBE',
    'XSD','ROBO','IYG','MOO','FIW','FTXN','IYJ','IHF','ARKW',
    'FSTA','FTXG','FXL','FDIS','RYH','MLPX','CGW','CQQQ','IDU',
    'IEO','ARKQ','SIL','IAI','DRIV','URNM','FXN','IYT','IGE',
    'IYM','ARKF','XHB','NANR','GRID','RHS','BUG','FIDU','FIVG',
    'IYC','XRT','SILJ','REMX','WCLD','FCG','RCD','IAT','GII',
    'KIE','CLOU','PSI','EMQQ','RYE','IAK','TPYP','IXG','XES',
    'ACES','FCOM','XHE','BBH','PNQI','IHAK','IPAY','FMAT',
    'PEJ','NXTG','IDRV','METV','XNTK','KBWP','MSOS','IEZ',
    'FINX','BLOK','PPH','RING','RYF','RGI','IHE','RYU','RTM',
    'FXD','CNRG','DRLL','KBWD','EXI','MXI','CHIQ','TECB',
    'PLDR','FXU','SNSR','PBJ','AMZA','FTRI','GINN','PJP',
    'PSCT','PSCH','IYZ','AIRR','FAN','IRBO','VEGI','PIO',
    'ARKX','RXI','ESPO','MJ','ATMP','IXP','PBE','XTN','MLPB',
    'GTEK','XSW','SGDM','ROBT','PTF','WOOD','XPH','SMOG',
    'GNOM','FTXO','UMI','IBUY','PXE','KARS','PTH','PRNT',
    'PSJ','SLVP','PYZ','DFNL','PKB','GSFP','HERO','BATT',
    'PFUT','SLX','TOLZ','AWAY','RTH','IDNA','PXI','MJUS',
    'SOCL','QABA','OGIG','JXI','HAP','BETZ','PRN','EDOC',
    'TDV','RAAX','ENFR','PSCE','FTXL','CTEC','IETC','AIQ',
    'PSL','FILL','DTEC','EWCO','EIPX','PXJ','QTUM','KCE',
    'LEGR','XHS','IGN','HTEC','GDOC','SBIO',
]

In [12]:
etf_histories = {}
US_BIZ_DAY = CustomBusinessDay(
    calendar=USFederalHolidayCalendar(),
    normalize=True
)
end_date: datetime = datetime(2023, 3, 9)
start_date: datetime = end_date - 250 * US_BIZ_DAY
print(end_date)
print(start_date)

2023-03-09 00:00:00
2022-03-09 00:00:00


In [13]:
for ticker in COUNTRY_ETF_TICKERS + SECTOR_ETF_TICKERS:
  etf_histories[ticker] = yf.Ticker(ticker).history(
      start=start_date.strftime('%Y-%m-%d'),
      end=end_date.strftime('%Y-%m-%d')
  )

In [14]:
len(etf_histories)

305

In [15]:
etf_histories['SBIO'].head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2022-03-09 00:00:00-05:00,32.270000,33.470001,32.259998,33.349998,14800,0.0,0.0,0.0
2022-03-10 00:00:00-05:00,32.919998,33.270000,32.575001,33.259998,7800,0.0,0.0,0.0
2022-03-11 00:00:00-05:00,33.610001,33.610001,32.110001,32.110001,11000,0.0,0.0,0.0
2022-03-14 00:00:00-04:00,32.110001,32.241001,30.450001,30.719999,18500,0.0,0.0,0.0
2022-03-15 00:00:00-04:00,30.780001,31.040001,30.459999,31.020000,19100,0.0,0.0,0.0


In [20]:
# store prices on disk for easier retrieval later
folder = '/Users/waleedzafar/Dropbox/WQU/Capstone/Week 7/Prices'
for ticker in tqdm(COUNTRY_ETF_TICKERS + SECTOR_ETF_TICKERS):
    etf_histories[ticker].to_csv(f'{folder}/{ticker}.csv')

100%|████████████████████████████████████████| 308/308 [00:01<00:00, 192.90it/s]


In [21]:
# create close price df for country etfs
df_country_etf_close = pd.DataFrame()
for ticker in tqdm(COUNTRY_ETF_TICKERS):
  df_country_etf_close = pd.concat([
      df_country_etf_close,
      etf_histories[ticker]['Close']
    ],
    axis=1
  )
  df_country_etf_close = df_country_etf_close.rename(columns={'Close': ticker})

df_country_etf_close.head()

100%|██████████████████████████████████████████| 44/44 [00:00<00:00, 472.82it/s]


,EWJ,MCHI,BBJP,VPL,KWEB,FXI,BBAX,EWT,EWY,AAXJ,...,DBJP,EWJV,EWM,PGJ,DFJ,MINV,ENZL,HEWJ,EPHE,FLCH
2022-03-09 00:00:00-05:00,59.042328,52.652187,48.511684,69.522171,29.540001,30.998035,49.669636,51.928959,69.408691,73.179573,...,45.001362,25.057539,24.463577,29.512945,64.847679,NaN,53.328178,24.581072,30.727924,20.821140
2022-03-10 00:00:00-05:00,58.795288,50.339962,48.344234,69.240082,26.660000,29.593906,50.011658,52.047672,67.314079,71.635323,...,44.961693,25.038069,24.463577,26.517010,64.964432,NaN,53.918423,24.553457,29.911268,19.992971
2022-03-11 00:00:00-05:00,57.846661,48.106113,47.516827,68.393791,24.000000,28.345795,49.698139,51.216675,66.306297,69.973045,...,44.713722,24.872576,24.317959,23.868284,64.390388,NaN,53.003551,24.415401,29.625933,19.213518
2022-03-14 00:00:00-04:00,57.925709,44.647568,47.595627,68.111694,21.190001,26.444374,49.109097,50.962284,66.019768,68.291100,...,45.209663,25.018600,24.327667,21.040993,64.302818,NaN,52.285427,24.629391,28.701044,17.859219
2022-03-15 00:00:00-04:00,58.874344,44.314449,48.432884,68.967720,22.180000,26.395618,49.308609,50.894447,66.750900,68.409134,...,45.933731,25.362240,24.385914,22.072706,65.587135,NaN,52.924850,25.071177,29.370113,17.674099


In [22]:
df_country_etf_close.shape

(251, 44)

In [23]:
df_sector_etf_close = pd.DataFrame()
for ticker in tqdm(SECTOR_ETF_TICKERS):
  df_sector_etf_close = pd.concat([
      df_sector_etf_close,
      etf_histories[ticker]['Close']
    ],
    axis=1
  )
  df_sector_etf_close = df_sector_etf_close.rename(columns={'Close': ticker})

df_sector_etf_close.head()

100%|████████████████████████████████████████| 264/264 [00:00<00:00, 753.50it/s]


,VGT,XLK,XLE,XLV,XLF,XLP,VHT,XLU,XLI,XLY,...,EIPX,PXJ,QTUM,KCE,LEGR,XHS,IGN,HTEC,GDOC,SBIO
2022-03-09 00:00:00-05:00,386.337463,147.798248,72.095360,127.971901,36.211895,71.123116,238.875168,68.157341,97.170357,167.332352,...,NaN,4.454967,47.700787,85.368248,37.597992,96.841087,70.041924,34.830002,31.774000,33.349998
2022-03-10 00:00:00-05:00,379.664062,145.213638,74.301285,127.922646,35.908249,70.450577,238.658173,68.613663,97.042572,168.521973,...,NaN,4.633563,47.109089,84.547966,36.809223,95.892845,69.433823,34.169998,31.642000,33.259998
2022-03-11 00:00:00-05:00,371.770905,142.658722,73.706642,126.967476,35.653591,69.631836,235.906342,68.380646,96.266068,165.280212,...,NaN,4.564109,46.113060,83.815575,36.672890,93.926468,68.157799,32.990002,30.952000,32.110001
2022-03-14 00:00:00-04:00,363.728973,140.014694,71.500725,127.863564,36.104153,70.002220,236.389664,68.302979,96.590431,162.524231,...,NaN,4.345825,45.166344,84.108528,36.818966,91.271378,66.203880,32.160000,30.427000,30.719999
2022-03-15 00:00:00-04:00,375.955475,144.718506,68.882378,130.413925,36.613491,71.571472,241.015472,69.176804,98.123764,167.966812,...,NaN,4.196996,46.379326,85.212006,37.266907,93.956413,68.167763,32.869999,31.000999,31.020000


In [24]:
df_sector_etf_close.shape

(251, 264)

In [25]:
# calculate daily price change pct
df_country_etf_returns = df_country_etf_close.pct_change()
df_country_etf_returns.head()

,EWJ,MCHI,BBJP,VPL,KWEB,FXI,BBAX,EWT,EWY,AAXJ,...,DBJP,EWJV,EWM,PGJ,DFJ,MINV,ENZL,HEWJ,EPHE,FLCH
2022-03-09 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-10 00:00:00-05:00,-0.004184,-0.043915,-0.003452,-0.004058,-0.097495,-0.045297,0.006886,0.002286,-0.030178,-0.021102,...,-0.000882,-0.000777,0.000000,-0.101513,0.001800,NaN,0.011068,-0.001123,-0.026577,-0.039775
2022-03-11 00:00:00-05:00,-0.016134,-0.044375,-0.017115,-0.012223,-0.099775,-0.042175,-0.006269,-0.015966,-0.014971,-0.023205,...,-0.005515,-0.006610,-0.005952,-0.099888,-0.008836,NaN,-0.016968,-0.005623,-0.009539,-0.038986
2022-03-14 00:00:00-04:00,0.001367,-0.071894,0.001658,-0.004125,-0.117083,-0.067079,-0.011852,-0.004967,-0.004321,-0.024037,...,0.011091,0.005871,0.000399,-0.118454,-0.001360,NaN,-0.013549,0.008765,-0.031219,-0.070487
2022-03-15 00:00:00-04:00,0.016377,-0.007461,0.017591,0.012568,0.046720,-0.001844,0.004063,-0.001331,0.011074,0.001728,...,0.016016,0.013735,0.002394,0.049033,0.019973,NaN,0.012229,0.017937,0.023312,-0.010365


In [26]:
df_sector_etf_returns = df_sector_etf_close.pct_change()
df_sector_etf_returns.head()

,VGT,XLK,XLE,XLV,XLF,XLP,VHT,XLU,XLI,XLY,...,EIPX,PXJ,QTUM,KCE,LEGR,XHS,IGN,HTEC,GDOC,SBIO
2022-03-09 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-10 00:00:00-05:00,-0.017274,-0.017487,0.030597,-0.000385,-0.008385,-0.009456,-0.000908,0.006695,-0.001315,0.007109,...,NaN,0.040089,-0.012404,-0.009609,-0.020979,-0.009792,-0.008682,-0.018949,-0.004154,-0.002699
2022-03-11 00:00:00-05:00,-0.020790,-0.017594,-0.008003,-0.007467,-0.007092,-0.011621,-0.011530,-0.003396,-0.008002,-0.019236,...,NaN,-0.014989,-0.021143,-0.008662,-0.003704,-0.020506,-0.018378,-0.034533,-0.021806,-0.034576
2022-03-14 00:00:00-04:00,-0.021631,-0.018534,-0.029928,0.007058,0.012637,0.005319,0.002049,-0.001136,0.003369,-0.016675,...,NaN,-0.047826,-0.020530,0.003495,0.003983,-0.028268,-0.028668,-0.025159,-0.016962,-0.043289
2022-03-15 00:00:00-04:00,0.033614,0.033595,-0.036620,0.019946,0.014107,0.022417,0.019569,0.012793,0.015875,0.033488,...,NaN,-0.034247,0.026856,0.013120,0.012166,0.029418,0.029664,0.022077,0.018865,0.009766


In [27]:
def calculate_annual_returns(tickers, df_prices) -> pd.DataFrame:
  df = pd.DataFrame(columns=['annualReturn'])
  for ticker in tickers:
    df.loc[ticker] = (df_prices[ticker].iloc[-1] - df_prices[ticker].iloc[0]) \
                          / df_prices[ticker].iloc[0]
  return df

In [28]:
# calculate annual returns
df_c_annual_returns = calculate_annual_returns(COUNTRY_ETF_TICKERS,
                                               df_country_etf_close)
df_s_annual_returns = calculate_annual_returns(SECTOR_ETF_TICKERS,
                                               df_sector_etf_close)

In [29]:
df_c_annual_returns.describe()

,annualReturn
count,43.000000
mean,-0.049163
std,0.104404
min,-0.380330
25%,-0.109470
50%,-0.044892
75%,-0.007048
max,0.276412


In [30]:
df_s_annual_returns.describe()

,annualReturn
count,262.000000
mean,-0.055964
std,0.135075
min,-0.640674
25%,-0.123848
50%,-0.035727
75%,0.022270
max,0.288450


In [31]:
df_c_annual_returns.query('annualReturn > 0')

,annualReturn
KWEB,0.006770
DXJ,0.276412
EWS,0.017171
CHIQ,0.002827
DBJP,0.194186
EWJV,0.028034
DFJ,0.028718
HEWJ,0.187906


In [32]:
df_s_annual_returns.query('annualReturn > 0.15')

,annualReturn
XLE,0.177607
VDE,0.178324
AMJ,0.181047
IXC,0.160844
IYE,0.171313
EUFN,0.170917
FENY,0.177536
IEO,0.178454
XES,0.190737
IEZ,0.175284


In [33]:
def get_underlying_name(ticker, category):
  '''
  Function to parse ETFs' underlying names fetched from Bloomberg
  '''
  name = ticker.replace(category, '').strip()
  if category == 'AU Equity':
    return f'{name}.AX'
  elif category == 'C1 Equity':
    return f'{name}.SS'
  elif category == 'C2 Equity':
    return f'{name}.SZ'
  elif category == 'CH Equity':
    if name.startswith('6'):
      return f'{name}.SS'
    elif name.startswith('3') or name.startswith('0'):
      return f'{name}.SZ'
  elif category == 'HK Equity':
    zeros = 4 - len(name)
    pre = '0' * zeros
    return f'{pre}{name}.HK'
  elif category == 'JP Equity':
    return f'{name}.T'
  elif category == 'KS Equity':
    return f'{name}.KS'
  elif category == 'TT Equity':
    return f'{name}.TW'
  elif category == 'US Equity':
    return name

  return None

In [34]:
df_underlyings = pd.read_csv('/Users/waleedzafar/Dropbox/WQU/Capstone/Week 7/ETF Underlying/ETF Underlyings.csv')
df_underlyings.head()

,Unnamed: 0,ETF Ticker,Underlying Ticker,Type
0,0,GXC,700 HK Equity,HK Equity
1,1,GXC,BABA US Equity,US Equity
2,2,GXC,3690 HK Equity,HK Equity
3,3,GXC,939 HK Equity,HK Equity
4,4,GXC,PDD US Equity,US Equity


In [35]:
df_underlyings.shape

(14750, 4)

In [36]:
df_underlyings.loc[:, 'Underlying'] = df_underlyings.apply(
    lambda row: get_underlying_name(row['Underlying Ticker'], row['Type']),
    axis=1
)

In [37]:
df_underlyings.head()

,Unnamed: 0,ETF Ticker,Underlying Ticker,Type,Underlying
0,0,GXC,700 HK Equity,HK Equity,0700.HK
1,1,GXC,BABA US Equity,US Equity,BABA
2,2,GXC,3690 HK Equity,HK Equity,3690.HK
3,3,GXC,939 HK Equity,HK Equity,0939.HK
4,4,GXC,PDD US Equity,US Equity,PDD


In [38]:
df_underlyings = df_underlyings.dropna()

In [39]:
df_underlyings.shape

(13442, 5)

In [40]:
underlying_tickers = df_underlyings['Underlying'].unique()

In [41]:
len(underlying_tickers)

4063

In [42]:
stock_histories = {}

In [43]:
for ticker in tqdm(underlying_tickers):
  stock_histories[ticker] = yf.Ticker(ticker).history(
      start=start_date.strftime('%Y-%m-%d'),
      end=end_date.strftime('%Y-%m-%d')
  )

  1%|▍                                        | 45/4063 [00:22<32:13,  2.08it/s]

SSIXX: Period '1mo' is invalid, must be one of ['1d', '5d']


 11%|████                                  | 434/4063 [03:49<3:29:42,  3.47s/it]

0921.HK: No data found for this date range, symbol may be delisted


 23%|█████████▏                              | 929/4063 [09:30<25:07,  2.08it/s]

...

 99%|██████████████████████████████████████▊| 4040/4063 [47:22<00:09,  2.34it/s]

Got error from yahoo api for ticker MMMAM.AX, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- MMMAM.AX: No timezone found, symbol may be delisted


 99%|██████████████████████████████████████▊| 4041/4063 [47:22<00:07,  2.76it/s]

Got error from yahoo api for ticker 0957001C.KS, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- 0957001C.KS: No timezone found, symbol may be delisted


100%|███████████████████████████████████████| 4063/4063 [47:33<00:00,  1.42it/s]


In [45]:
# store prices on disk for easier retrieval later
folder = '/Users/waleedzafar/Dropbox/WQU/Capstone/Week 7/Prices'
for ticker in tqdm(underlying_tickers):
    df = stock_histories[ticker]
    if isinstance(df, pd.DataFrame) and not df.empty:
        df.to_csv(f'{folder}/{ticker}.csv')

100%|██████████████████████████████████████| 4063/4063 [00:25<00:00, 161.54it/s]


In [46]:
df_underlying_close = pd.DataFrame()
for ticker in underlying_tickers:
  df = stock_histories[ticker]['Close']
  try:
    df.index = df.index.tz_localize(None)
    df_underlying_close = pd.concat([
        df_underlying_close,
        df
      ],
      axis=1
    )
    df_underlying_close = df_underlying_close.rename(columns={'Close': ticker})
  except:
    try:
      df.index = pd.to_datetime(df.index)
      df.index = df.index.tz_localize(None)
      df_underlying_close = pd.concat([
          df_underlying_close,
          df
      ],
      axis=1
      )
      df_underlying_close = df_underlying_close.rename(
          columns={'Close': ticker}
      )
    except:
      pass

df_underlying_close.head()

,0700.HK,BABA,3690.HK,0939.HK,PDD,2318.HK,JD,BIDU,1398.HK,NTES,...,688561.SS,688169.SS,688303.SS,603195.SS,600606.SS,688065.SS,605499.SS,601236.SS,300979.SZ,001289.SZ
2022-03-09 00:00:00,382.998688,100.930000,148.500000,5.106776,43.340000,50.223202,60.864376,144.229996,4.177830,86.721725,...,60.349998,377.489716,59.258842,132.763718,3.700000,79.672607,153.797195,8.914510,73.694801,29.837063
2022-03-10 00:00:00,383.197784,92.919998,144.199997,5.199290,35.759998,51.823566,51.227516,135.160004,4.233411,80.382950,...,62.200001,373.242126,62.120773,133.245514,3.709090,78.430054,155.834518,8.845251,71.989906,29.409824
2022-03-11 00:00:00,366.078278,86.709999,135.399994,5.199290,32.130001,51.635288,46.808998,118.919998,4.242674,77.786453,...,63.529999,366.934631,62.665432,134.858109,3.709090,79.715202,154.519165,9.191542,70.166748,28.167856
2022-03-14 00:00:00,330.246796,77.760002,112.599998,5.208542,25.530001,49.940784,41.883270,108.970001,4.261201,70.351059,...,60.480000,361.337463,58.209137,133.432343,3.618181,77.173294,145.479843,9.033237,66.520447,27.790297
2022-03-15 00:00:00,296.605042,76.760002,106.000000,5.079022,27.309999,43.633469,44.848461,108.379997,4.103722,73.016388,...,57.040001,343.352631,57.020794,125.890518,3.372727,73.360428,140.979950,8.855145,62.755882,25.832954


In [47]:
df_underlying_close.shape

(261, 4063)

In [48]:
df_underlying_returns = df_underlying_close.pct_change()
df_underlying_returns.head()

,0700.HK,BABA,3690.HK,0939.HK,PDD,2318.HK,JD,BIDU,1398.HK,NTES,...,688561.SS,688169.SS,688303.SS,603195.SS,600606.SS,688065.SS,605499.SS,601236.SS,300979.SZ,001289.SZ
2022-03-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-10 00:00:00,0.000520,-0.079362,-0.028956,0.018116,-0.174896,0.031865,-0.158333,-0.062886,0.013304,-0.073093,...,0.030655,-0.011252,0.048295,0.003629,0.002457,-0.015596,0.013247,-0.007769,-0.023135,-0.014319
2022-03-11 00:00:00,-0.044675,-0.066832,-0.061026,0.000000,-0.101510,-0.003633,-0.086253,-0.120154,0.002188,-0.032302,...,0.021383,-0.016899,0.008768,0.012102,0.000000,0.016386,-0.008441,0.039150,-0.025325,-0.042230
2022-03-14 00:00:00,-0.097879,-0.103218,-0.168390,0.001779,-0.205416,-0.032817,-0.105230,-0.083670,0.004367,-0.095587,...,-0.048009,-0.015254,-0.071112,-0.010572,-0.024510,-0.031887,-0.058500,-0.017223,-0.051966,-0.013404
2022-03-15 00:00:00,-0.101869,-0.012860,-0.058615,-0.024867,0.069722,-0.126296,0.070797,-0.005414,-0.036956,0.037886,...,-0.056878,-0.049773,-0.020415,-0.056522,-0.067839,-0.049407,-0.030931,-0.019715,-0.056593,-0.070433


In [49]:
# standardize returns' indices
df_country_etf_returns.index = pd.to_datetime(df_country_etf_returns.index)
df_sector_etf_returns.index = pd.to_datetime(df_sector_etf_returns.index)
df_underlying_returns.index = pd.to_datetime(df_underlying_returns.index)
df_country_etf_returns.index = df_country_etf_returns.index.tz_localize(None)
df_sector_etf_returns.index = df_sector_etf_returns.index.tz_localize(None)
df_underlying_returns.index = df_underlying_returns.index.tz_localize(None)

In [50]:
# calculate monthly returns
def calculate_monthly_returns(df):
  df_m = df.resample('M').agg(lambda x: (x+1).prod() - 1)
  return df_m

In [51]:
df_c_monthly_rets = calculate_monthly_returns(df_country_etf_returns)
df_s_monthly_rets = calculate_monthly_returns(df_sector_etf_returns)
df_u_monthly_rets = calculate_monthly_returns(df_underlying_returns)

In [52]:
print(df_c_monthly_rets.shape)
print(df_s_monthly_rets.shape)
print(df_u_monthly_rets.shape)

(13, 44)
(13, 264)
(13, 4063)


In [53]:
df_c_monthly_rets.head()

,EWJ,MCHI,BBJP,VPL,KWEB,FXI,BBAX,EWT,EWY,AAXJ,...,DBJP,EWJV,EWM,PGJ,DFJ,MINV,ENZL,HEWJ,EPHE,FLCH
2022-03-31,0.031130,-0.015817,0.030254,0.036823,-0.034868,0.005662,0.074457,0.003429,0.013950,0.008468,...,0.080450,0.037762,0.016270,-0.025882,-0.011103,0.000000,0.014758,0.083404,-0.004163,-0.013617
2022-04-30,-0.080993,-0.045944,-0.080607,-0.069142,-0.008418,-0.032218,-0.065253,-0.090155,-0.064439,-0.054778,...,-0.016116,-0.058850,-0.031238,-0.049689,-0.063268,0.000000,-0.091801,-0.018403,-0.072990,-0.051663
2022-05-31,0.017308,0.026358,0.017578,0.011751,0.030421,0.029735,0.005769,0.020569,0.021008,0.010857,...,0.011818,0.007558,-0.025796,0.027959,0.011338,0.000000,-0.032226,0.011091,0.018730,0.022511
2022-06-30,-0.073902,0.083678,-0.071624,-0.081823,0.124614,0.068913,-0.069468,-0.116369,-0.144768,-0.030244,...,-0.023165,-0.053277,-0.073517,0.154078,-0.033515,0.000000,-0.069630,-0.023244,-0.125846,0.086531
2022-07-31,0.062843,-0.109658,0.062628,0.053403,-0.134615,-0.104099,0.038583,0.017255,0.040557,-0.017341,...,0.046089,0.043644,0.029157,-0.108622,0.053252,0.004946,0.081628,0.049577,0.022065,-0.104356


In [55]:
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [56]:
warnings.simplefilter('ignore', ConvergenceWarning)

In [57]:
# predict next month's return
df_monthly_rets_pred = pd.DataFrame(columns=['PredictedReturn'])

In [64]:
def get_monthly_return_prediction(df, df_returns, max_i=5, max_j=5, max_k=5):
    for col in tqdm(df.columns):
        data = pd.DataFrame(df[col])
        min_aic_order = None
        min_aic = 10000000
        for i in range(1, max_i):
            for j in range(1, max_j):
                for k in range(max_k):
                    try:
                        model = ARIMA(data, order=(i, j, k)).fit()
                        aic = model.aic
                        if aic < min_aic:
                            min_aic = aic
                            min_aic_order = (i, j, k)
                    except:
                        pass
        if min_aic_order:
            model = ARIMA(data, order=min_aic_order).fit()
            pred = model.predict()[-1]
            df_returns.loc[col] = pred

In [ ]:
get_monthly_return_prediction(df_c_monthly_rets, df_monthly_rets_pred)

In [60]:
df_monthly_rets_pred.shape

(44, 1)

In [62]:
df_monthly_rets_pred.transpose()

,EWJ,MCHI,BBJP,VPL,KWEB,FXI,BBAX,EWT,EWY,AAXJ,...,DBJP,EWJV,EWM,PGJ,DFJ,MINV,ENZL,HEWJ,EPHE,FLCH
PredictedReturn,0.024023,0.014215,0.022955,0.0095,0.036447,0.017948,0.016625,0.032875,0.00866,0.003853,...,0.01785,0.007432,-0.009782,0.041608,0.059691,0.026143,-0.004399,0.019222,0.00738,0.013483


In [65]:
# including only country ETFs
df_monthly_rets_pred.describe()

,PredictedReturn
count,44.000000
mean,0.006715
std,0.031686
min,-0.109043
25%,0.003084
50%,0.011254
75%,0.021359
max,0.059691


In [66]:
df_s_monthly_rets_pred = pd.DataFrame(columns=['PredictedReturn'])

In [ ]:
get_monthly_return_prediction(df_s_monthly_rets, df_s_monthly_rets_pred,
                             max_i=4, max_j=4, max_k=4)

In [73]:
df_s_monthly_rets_pred.transpose()

,VGT,XLK,XLE,XLV,XLF,XLP,VHT,XLU,XLI,XLY,...,EIPX,PXJ,QTUM,KCE,LEGR,XHS,IGN,HTEC,GDOC,SBIO
PredictedReturn,0.013411,0.017046,0.044608,0.001566,-0.000058,0.010215,0.004276,0.035816,0.008421,-0.004344,...,0.018416,0.041317,0.026461,0.071346,0.029237,0.000439,0.008563,-0.000737,0.036097,-0.01784


In [74]:
# predicted monthly return for sector ETFs
# trained by selecting ARIMA model with min AIC
# considering all ARIMA models in range 
# (1, 4) for p, d and (0, 4) for q
df_s_monthly_rets_pred.describe()

,PredictedReturn
count,264.000000
mean,0.014564
std,0.026834
min,-0.076159
25%,-0.000640
50%,0.013320
75%,0.030426
max,0.091423


In [75]:
import os

In [76]:
# store predicted monthly returns to CSV
folder = f'{os.getcwd()}/data'
df_monthly_rets_pred.to_csv(f'{folder}/c_pred_monthly_rets.csv')
df_s_monthly_rets_pred.to_csv(f'{folder}/s_pred_monthly_rets.csv')

In [77]:
df_u_monthly_rets_pred = pd.DataFrame(columns=['PredictedReturn'])

In [ ]:
get_monthly_return_prediction(df_u_monthly_rets, df_u_monthly_rets_pred,
                             max_i=3, max_j=2, max_k=3)

In [79]:
df_u_monthly_rets_pred.shape

(4063, 1)

In [80]:
df_u_monthly_rets_pred.transpose()

,0700.HK,BABA,3690.HK,0939.HK,PDD,2318.HK,JD,BIDU,1398.HK,NTES,...,688561.SS,688169.SS,688303.SS,603195.SS,600606.SS,688065.SS,605499.SS,601236.SS,300979.SZ,001289.SZ
PredictedReturn,0.004156,0.076791,0.003559,-0.001251,0.110653,0.031376,0.08309,0.023505,-0.006254,0.325763,...,0.098126,-0.141324,-0.006657,0.013007,0.033549,-0.189675,0.089912,-0.22854,0.017674,-0.164838


In [81]:
df_u_monthly_rets_pred.describe()

,PredictedReturn
count,4063.000000
mean,0.028081
std,0.110659
min,-0.476742
25%,-0.011229
50%,0.012084
75%,0.056814
max,1.494197


In [82]:
folder = f'{os.getcwd()}/data'
df_u_monthly_rets_pred.to_csv(f'{folder}/u_pred_monthly_rets.csv')

In [85]:
df_u_monthly_rets_pred.sort_values(by='PredictedReturn', ascending=False).head(15)

,PredictedReturn
3660.T,1.494197
DADA,1.437514
GCPEF,1.411325
GOTU,1.184315
6338.T,1.100221
2767.T,1.004610
000756.SZ,0.937768
LX,0.932689
4776.T,0.864831
NEGG,0.803300


In [87]:
# calculate monthly returns
def calculate_monthly_returns(df):
  df_m = df.resample('M').agg(lambda x: (x+1).prod() - 1)
  return df_m

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2022-03-09 00:00:00-05:00,58.716235,59.131262,58.409909,59.042328,7410400,0.0,0.0,0.0
2022-03-10 00:00:00-05:00,58.933629,59.170788,58.558129,58.795288,15922900,0.0,0.0,0.0
2022-03-11 00:00:00-05:00,58.894104,58.943515,57.777490,57.846661,5751100,0.0,0.0,0.0
2022-03-14 00:00:00-04:00,58.212275,58.488957,57.777488,57.925709,4843700,0.0,0.0,0.0
2022-03-15 00:00:00-04:00,58.232041,59.022565,58.143107,58.874344,11392200,0.0,0.0,0.0


In [92]:
def normalize_datetime_index(df):
    ndf = df
    try:
        ndf.index = pd.to_datetime(ndf.index)
        ndf.index = ndf.index.tz_localize(None)
    except:
        pass
    return ndf

In [96]:
df_c_volume = pd.DataFrame()
for ticker in tqdm(COUNTRY_ETF_TICKERS):
    df_c_volume = pd.concat([
        df_c_volume,
        normalize_datetime_index(
            etf_histories[ticker]['Volume']
        )
    ],
    axis=1
    )
    df_c_volume = df_c_volume.rename(columns={'Volume': ticker})
    
print(df_c_volume.shape)
df_c_volume.head()

100%|██████████████████████████████████████████| 44/44 [00:00<00:00, 477.60it/s]

(251, 44)


,EWJ,MCHI,BBJP,VPL,KWEB,FXI,BBAX,EWT,EWY,AAXJ,...,DBJP,EWJV,EWM,PGJ,DFJ,MINV,ENZL,HEWJ,EPHE,FLCH
2022-03-09 00:00:00,7410400,5066500,377900,3669900,16631600,40442000,225100,3764700,5768500,1901500,...,1300,40800,1166300,51700,50700,NaN,13500,496500,660500,28000
2022-03-10 00:00:00,15922900,5610300,775600,1536400,31395400,59032900,456800,5670800,8852200,1847500,...,18100,36700,677500,199400,47500,NaN,14900,174500,1423600,40100
2022-03-11 00:00:00,5751100,7009600,1258400,1139100,37158000,54458400,394900,5459600,5444800,1766600,...,16900,37900,859700,134800,18600,NaN,6100,180800,424300,480100
2022-03-14 00:00:00,4843700,9771000,768700,976200,56502200,88118800,635200,4043300,8717800,1882700,...,16700,40400,294700,422900,5400,NaN,9500,342400,312100,88900
2022-03-15 00:00:00,11392200,13852200,1289700,1506200,65256200,116703800,567400,4992100,5357800,5299300,...,6700,19000,566700,319200,21200,NaN,6600,438300,456400,90600


In [97]:
df_s_volume = pd.DataFrame()
for ticker in tqdm(SECTOR_ETF_TICKERS):
    df_s_volume = pd.concat([
        df_s_volume,
        normalize_datetime_index(
            etf_histories[ticker]['Volume']
        )
    ],
    axis=1
    )
    df_s_volume = df_s_volume.rename(columns={'Volume': ticker})

print(df_s_volume.shape)
df_s_volume.head()

100%|████████████████████████████████████████| 264/264 [00:00<00:00, 783.71it/s]

(251, 264)


,VGT,XLK,XLE,XLV,XLF,XLP,VHT,XLU,XLI,XLY,...,EIPX,PXJ,QTUM,KCE,LEGR,XHS,IGN,HTEC,GDOC,SBIO
2022-03-09 00:00:00,664500,17663700,87018200,15576500,88716400,18254000,178100,25476900,18757400,15969100,...,NaN,1120400,17100,22100,13500,4000,11400,27700,1300,14800
2022-03-10 00:00:00,988300,13724200,68901300,10708800,71547300,21892400,128200,17019900,13317200,12927400,...,NaN,952800,12300,51300,10100,3500,126000,27100,1700,7800
2022-03-11 00:00:00,796300,10728900,45918900,9869100,76788200,14276500,167500,12494200,10695200,12078200,...,NaN,474200,15100,13700,22400,3900,31700,30300,2500,11000
2022-03-14 00:00:00,1091500,13926200,50745000,14373200,70969700,18438900,179300,20502300,12988000,15174600,...,NaN,394700,50400,47100,8800,3700,21300,93800,1500,18500
2022-03-15 00:00:00,508600,11496900,62362200,14374100,51828700,17078100,180500,14502200,11204900,13560200,...,NaN,482500,12000,18700,16100,3100,43400,43800,100,19100


In [95]:
df_u_volume = pd.DataFrame()
for ticker in tqdm(underlying_tickers):
    df_u_volume = pd.concat([
        df_u_volume,
        normalize_datetime_index(
            stock_histories[ticker]['Volume']
        )
    ],
    axis=1
    )
    df_u_volume = df_u_volume.rename(columns={'Volume': ticker})

print(df_u_volume.shape)
df_u_volume.head()

100%|██████████████████████████████████████| 4063/4063 [00:11<00:00, 344.39it/s]

(261, 4063)


,0700.HK,BABA,3690.HK,0939.HK,PDD,2318.HK,JD,BIDU,1398.HK,NTES,...,688561.SS,688169.SS,688303.SS,603195.SS,600606.SS,688065.SS,605499.SS,601236.SS,300979.SZ,001289.SZ
2022-03-09 00:00:00,33209737.0,20853600.0,40551427.0,436940704.0,7731200.0,38150491.0,9487300.0,3303000.0,294831097.0,3654700.0,...,3113310.0,818528.0,7539227.0,860472.0,54096126.0,1775545.0,1044928.0,14217675.0,1658895.0,22259464.0
2022-03-10 00:00:00,24062703.0,37671200.0,35919402.0,328950220.0,18296900.0,27457662.0,55348100.0,4452300.0,170116737.0,5597700.0,...,2292586.0,1091686.0,11634584.0,678855.0,35343896.0,1579177.0,796417.0,8656122.0,1508246.0,20755385.0
2022-03-11 00:00:00,48435608.0,39341800.0,72625389.0,278126910.0,17274400.0,34598291.0,32963000.0,7837300.0,243770862.0,4742700.0,...,2334723.0,863513.0,7668172.0,636000.0,50465649.0,1159095.0,780754.0,17118222.0,2289422.0,14199023.0
2022-03-14 00:00:00,66910165.0,69992600.0,103004664.0,499697978.0,32943500.0,39426431.0,47477300.0,13559900.0,338753089.0,6159000.0,...,2425185.0,791243.0,7061263.0,607813.0,36354450.0,1198255.0,1089759.0,10671788.0,1698430.0,10689606.0
2022-03-15 00:00:00,118398276.0,86209300.0,105044140.0,566144787.0,56774300.0,114500513.0,39194500.0,9396400.0,517738144.0,4652000.0,...,3388645.0,696236.0,5060311.0,1326037.0,69402976.0,1504970.0,872903.0,14521967.0,1931991.0,13197613.0


In [98]:
df_underlyings.head()

,Unnamed: 0,ETF Ticker,Underlying Ticker,Type,Underlying
0,0,GXC,700 HK Equity,HK Equity,0700.HK
1,1,GXC,BABA US Equity,US Equity,BABA
2,2,GXC,3690 HK Equity,HK Equity,3690.HK
3,3,GXC,939 HK Equity,HK Equity,0939.HK
4,4,GXC,PDD US Equity,US Equity,PDD


In [99]:
df_underlyings.loc[:, 'CountryETF'] = df_underlyings['ETF Ticker'].apply(
    lambda x: True if x in COUNTRY_ETF_TICKERS else False
)
df_underlyings.loc[:, 'SectorETF'] = df_underlyings['ETF Ticker'].apply(
    lambda x: True if x in SECTOR_ETF_TICKERS else False
)

In [162]:
df_underlyings['ETF Ticker'].unique()

array(['GXC', 'AAXJ', 'DXJ', 'DBJP', 'PGJ', 'EWJ', 'EEMA', 'GMF', 'EWT',
       'EWA', 'AIA', 'EWY', 'DFJ', 'FLJP', 'IPAC', 'BBJP', 'BBAX', 'CXSE',
       'KBA', 'MINV', 'MCHI', 'EPP', 'FLCH', 'EWJV', 'FXI', 'VPL', 'FLKR',
       'CNYA', 'ASHR', 'EWH'], dtype=object)

In [100]:
df_underlyings.head()

,Unnamed: 0,ETF Ticker,Underlying Ticker,Type,Underlying,CountryETF,SectorETF
0,0,GXC,700 HK Equity,HK Equity,0700.HK,True,False
1,1,GXC,BABA US Equity,US Equity,BABA,True,False
2,2,GXC,3690 HK Equity,HK Equity,3690.HK,True,False
3,3,GXC,939 HK Equity,HK Equity,0939.HK,True,False
4,4,GXC,PDD US Equity,US Equity,PDD,True,False


In [105]:
df_underlyings = df_underlyings.drop(
    df_underlyings.query('CountryETF == True & SectorETF == True').index,
    axis=0
)

In [106]:
df_underlyings.shape

(13211, 7)

In [107]:
print(df_c_volume.shape)
print(df_s_volume.shape)
print(df_u_volume.shape)

(251, 44)
(251, 264)
(261, 4063)


In [108]:
# get monthly volume
def calculate_monthly_volume(df):
    df_m = df.resample('M').agg(np.sum)
    return df_m

In [110]:
df_c_monthly_volume = calculate_monthly_volume(
    normalize_datetime_index(df_c_volume)
)

print(df_c_monthly_volume.shape)
df_c_monthly_volume.head()

(13, 44)


,EWJ,MCHI,BBJP,VPL,KWEB,FXI,BBAX,EWT,EWY,AAXJ,...,DBJP,EWJV,EWM,PGJ,DFJ,MINV,ENZL,HEWJ,EPHE,FLCH
2022-03-31,123338300,154003100,12096600,19023700,666988700,1128494700,6400900,83911000,78836000,31974000,...,325000,486300,10173200,5251300,259200,0.0,172400,3431500,6009100,1176700
2022-04-30,145205600,147909200,9944100,21554900,412189700,852863900,5808400,104036600,57609500,38543600,...,195800,592500,17722500,2511000,272800,0.0,283000,6031900,1695300,1043900
2022-05-31,150059700,134359900,16076400,20474300,564877400,826921700,7603700,91289800,82442300,43256800,...,275200,540100,14266300,2705700,371600,0.0,190000,5407700,3490300,474100
2022-06-30,113213500,158772800,17200200,19449400,555692000,843281800,7166300,95209700,74229100,30029200,...,294700,236400,9342700,3320600,354200,0.0,197700,5308200,3496800,488500
2022-07-31,75119000,110019600,9613500,19867700,335251900,646441100,4898200,60012100,67507800,22700200,...,350300,167200,6285400,1682400,218700,51400.0,365100,4421600,1338400,538200


In [111]:
df_s_monthly_volume = calculate_monthly_volume(
    normalize_datetime_index(df_s_volume)
)

print(df_s_monthly_volume.shape)
df_s_monthly_volume.head()

(13, 264)


,VGT,XLK,XLE,XLV,XLF,XLP,VHT,XLU,XLI,XLY,...,EIPX,PXJ,QTUM,KCE,LEGR,XHS,IGN,HTEC,GDOC,SBIO
2022-03-31,10987400,182762600,716968100,193077900,1043280200,262761400,4818500,288979900,191532700,197147500,...,0.0,8644400,256800,362900,279900,86100,1325400,529900,192300,308100
2022-04-30,13749200,235094300,645446800,320912000,1329232900,374221800,7092200,413997200,240799900,196284700,...,0.0,16476400,341900,366100,289100,106900,608200,678600,43900,587400
2022-05-31,23497500,268894200,783477100,292279000,1678540800,597327600,9820100,433573200,321569900,194840400,...,0.0,11253800,323100,572700,302200,53900,314300,809400,47400,649900
2022-06-30,15902500,186992500,775796100,198570500,1313433900,311767800,7182500,343124700,201916900,141485200,...,0.0,20072200,414500,185600,260000,71300,252300,480600,174200,581300
2022-07-31,9997400,131312800,557473400,164183900,836301800,207865300,3875400,235551200,155201800,102720200,...,0.0,7459400,311700,165600,217100,306300,181100,284800,43300,379200


In [112]:
df_u_monthly_volume = calculate_monthly_volume(
    normalize_datetime_index(df_u_volume)
)

print(df_u_monthly_volume.shape)
df_u_monthly_volume.head()

(13, 4063)


,0700.HK,BABA,3690.HK,0939.HK,PDD,2318.HK,JD,BIDU,1398.HK,NTES,...,688561.SS,688169.SS,688303.SS,603195.SS,600606.SS,688065.SS,605499.SS,601236.SS,300979.SZ,001289.SZ
2022-03-31,784731749.0,1.023127e+09,1.020211e+09,6.186814e+09,599781100.0,712295989.0,471624700.0,122697400.0,4.355673e+09,68128500.0,...,39344743.0,13193031.0,94324657.0,11348410.0,2.298040e+09,22882205.0,14699635.0,3.992188e+08,20683794.0,184840975.0
2022-04-30,396316513.0,5.408851e+08,6.170945e+08,4.683171e+09,324772500.0,490727892.0,215210200.0,71580600.0,3.255832e+09,37491600.0,...,46252663.0,11149455.0,101909582.0,11490651.0,3.021766e+09,30687570.0,16754479.0,9.412351e+08,14414041.0,104368381.0
2022-05-31,523342151.0,4.826922e+08,7.435653e+08,5.074850e+09,259146200.0,577206201.0,298611400.0,69765000.0,3.139446e+09,45624500.0,...,47162034.0,10979661.0,131058641.0,11598813.0,2.230893e+09,32132320.0,21441315.0,3.962861e+08,25553786.0,119863591.0
2022-06-30,568117466.0,7.216166e+08,7.482574e+08,6.357927e+09,332852300.0,647735331.0,290304100.0,72292400.0,4.092837e+09,49782400.0,...,70835686.0,16188999.0,299627052.0,14040279.0,1.868261e+09,64477431.0,20677572.0,1.123206e+09,29019130.0,124617590.0
2022-07-31,404917763.0,5.046133e+08,3.964437e+08,4.077032e+09,177256600.0,475308323.0,121077500.0,41560400.0,2.933923e+09,36436100.0,...,41065483.0,19466287.0,277471755.0,9716312.0,1.366010e+09,41087316.0,17433622.0,4.953340e+08,39441988.0,147281539.0


In [115]:
# calculate annual volume
df_c_annual_volume = df_c_volume.sum()
print(df_c_annual_volume.shape)
df_c_annual_volume.describe()

(44,)


count    4.400000e+01
mean     5.961987e+08
std      1.722976e+09
min      3.972700e+06
25%      2.312890e+07
50%      6.137510e+07
75%      1.822823e+08
max      1.005144e+10
dtype: float64

In [116]:
df_s_annual_volume = df_s_volume.sum()
print(df_s_annual_volume.shape)
df_s_annual_volume.describe()

(264,)


count    2.640000e+02
mean     3.426421e+08
std      1.182069e+09
min      6.019000e+05
25%      1.064702e+07
50%      3.048700e+07
75%      1.185149e+08
max      1.238964e+10
dtype: float64

In [117]:
df_u_annual_volume = df_u_volume.sum()
print(df_u_annual_volume.shape)
df_u_annual_volume.describe()

(4063,)


count    4.063000e+03
mean     2.344526e+09
std      6.687033e+09
min      0.000000e+00
25%      2.993852e+07
50%      2.449560e+08
75%      1.919819e+09
max      2.050973e+11
dtype: float64

In [118]:
# calculate annual returns
def calculate_annual_returns(df):
    df_m = df.resample('Y').agg(lambda x: (x+1).prod() - 1)
    return df_m

In [120]:
print(df_country_etf_returns.shape)
print(df_sector_etf_returns.shape)
print(df_underlying_returns.shape)

(251, 44)
(251, 264)
(261, 4063)


In [124]:
df_c_annual_returns = calculate_annual_returns(
    normalize_datetime_index(df_country_etf_returns)
)
print(df_c_annual_returns.shape)
df_c_annual_returns.transpose().describe()

(2, 44)


,2022-12-31,2023-12-31
count,44.000000,44.000000
mean,-0.075201,0.028316
std,0.086404,0.040192
min,-0.375052,-0.049475
25%,-0.125252,0.000546
50%,-0.074181,0.022939
75%,-0.022276,0.048326
max,0.123194,0.136413


In [125]:
df_s_annual_returns = calculate_annual_returns(
    normalize_datetime_index(df_sector_etf_returns)
)
print(df_s_annual_returns.shape)
df_s_annual_returns.transpose().describe()

(2, 264)


,2022-12-31,2023-12-31
count,264.000000,264.000000
mean,-0.097231,0.052831
std,0.148744,0.075191
min,-0.631718,-0.107109
25%,-0.202349,-0.005998
50%,-0.085292,0.050827
75%,0.013911,0.108925
max,0.230533,0.317250


In [126]:
df_u_annual_returns = calculate_annual_returns(
    normalize_datetime_index(df_underlying_returns)
)
print(df_u_annual_returns.shape)
df_u_annual_returns.transpose().describe()

(2, 4063)


,2022-12-31,2023-12-31
count,4063.000000,4063.000000
mean,-0.029937,0.101015
std,0.392577,0.234788
min,-0.987519,-0.706359
25%,-0.211255,-0.002351
50%,-0.052806,0.071638
75%,0.085475,0.177268
max,11.899262,4.409317


In [145]:
df_underlyings.head()

,Unnamed: 0,ETF Ticker,Underlying Ticker,Type,Underlying,CountryETF,SectorETF
0,0,GXC,700 HK Equity,HK Equity,0700.HK,True,False
1,1,GXC,BABA US Equity,US Equity,BABA,True,False
2,2,GXC,3690 HK Equity,HK Equity,3690.HK,True,False
3,3,GXC,939 HK Equity,HK Equity,0939.HK,True,False
4,4,GXC,PDD US Equity,US Equity,PDD,True,False


In [177]:
df_u_monthly_rets_pred.head()

,PredictedReturn
0700.HK,0.004156
BABA,0.076791
3690.HK,0.003559
0939.HK,-0.001251
PDD,0.110653


In [192]:
def build_ticker_features_annual():
    df = pd.DataFrame(columns=[
        'annual_volume', 'c_etf_annual_volume', 'c_etf_annual_return', 
        'annual_return'
    ])
    for ticker in tqdm(underlying_tickers):
        df_map = df_underlyings[
            df_underlyings['Underlying'] == ticker
        ]
        if not df_map.empty:
            df_c = df_map.query('CountryETF == True')
            if not df_c.empty:
                # get country ETF name
                c_etf = df_c['ETF Ticker'].iloc[0]
                
                # get underlying annual volume
                annual_volume = df_u_annual_volume.loc[ticker]
                
                # get underlying annual return
                annual_return = df_u_annual_returns[ticker].iloc[-1]
                
                # get country etf annual volume
                c_annual_volume = df_c_annual_volume.loc[c_etf]
                
                # get country etf annual return
                c_annual_return = df_c_annual_returns[c_etf].iloc[-1]
                
                df.loc[ticker] = [annual_volume, c_annual_volume, c_annual_return, 
                                  annual_return]
                
    return df
                

In [197]:
u_data = build_ticker_features_annual()

100%|██████████████████████████████████████| 4063/4063 [00:08<00:00, 461.79it/s]


In [198]:
u_data.shape

(4058, 4)

In [199]:
u_data.head()

,annual_volume,c_etf_annual_volume,c_etf_annual_return,annual_return
0700.HK,6.820517e+09,46933800.0,0.022205,0.102053
BABA,6.720420e+09,46933800.0,0.022205,0.320875
3690.HK,8.381797e+09,46933800.0,0.022205,-0.244991
0939.HK,6.170902e+10,46933800.0,0.022205,0.024540
PDD,3.405556e+09,46933800.0,0.022205,0.291938


In [209]:
def build_ticker_features(ticker):
    df = pd.DataFrame(columns=['ticker', 'annual_volume', 'annual_return',
                              'c_etf_annual_volume', 'c_etf_annual_return'])
    try:
        df = pd.concat([
            df,
            df_u_monthly_rets[ticker]
        ],
        axis=1)
        df = df.rename(columns={ticker: 'monthly_return'})
        df = pd.concat([
            df,
            df_u_monthly_volume[ticker]
        ],
        axis=1)
        df = df.rename(columns={ticker: 'monthly_volume'})

        annual_volume = u_data['annual_volume'].loc[ticker]
        annual_return = u_data['annual_return'].loc[ticker]
        c_etf_annual_volume = u_data['c_etf_annual_volume'].loc[ticker]
        c_etf_annual_return = u_data['c_etf_annual_return'].loc[ticker]

        df['ticker'] = ticker
        df['annual_volume'] = annual_volume
        df['annual_return'] = annual_return
        df['c_etf_annual_volume'] = c_etf_annual_volume
        df['c_etf_annual_return'] = c_etf_annual_return
    except:
        pass
    
    return df

In [210]:
df_u_features = pd.concat(
    [build_ticker_features(x) for x in tqdm(underlying_tickers)],
    axis=0
)


100%|██████████████████████████████████████| 4063/4063 [00:08<00:00, 464.90it/s]


In [211]:
df_u_features.head()

,ticker,annual_volume,annual_return,c_etf_annual_volume,c_etf_annual_return,monthly_return,monthly_volume
2022-03-31,0700.HK,6.820517e+09,0.102053,46933800.0,0.022205,-0.027547,784731749.0
2022-04-30,0700.HK,6.820517e+09,0.102053,46933800.0,0.022205,0.008552,396316513.0
2022-05-31,0700.HK,6.820517e+09,0.102053,46933800.0,0.022205,-0.035229,523342151.0
2022-06-30,0700.HK,6.820517e+09,0.102053,46933800.0,0.022205,-0.022075,568117466.0
2022-07-31,0700.HK,6.820517e+09,0.102053,46933800.0,0.022205,-0.134312,404917763.0


In [212]:
df_u_features.shape

(52819, 7)

In [213]:
# train regression models using the monthly_return as the independent variable